https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification


In [1]:
! pip install loguru==0.7.2  evaluate==0.4.1 wandb -q 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/documents-raw/documents_raw.csv


In [3]:
from datasets import ClassLabel, Dataset, Features, Image, Value, Sequence, DatasetDict
label_dic={"business":0,"entertainment":1,"food":2,"graphics":3,"historical":4,"medical":5,"politics":6,"space":7,"sport":8,"technologie":9}


Clabels= ClassLabel(num_classes=10, names=["business","entertainment","food","graphics","historical","medical","politics","space","sport","technologie"])


df = pd.read_csv("/kaggle/input/documents-raw/documents_raw.csv")
df = df[["Class", "Document"]]
df['label2']= df['Class'].map(label_dic)

In [4]:
df.head(2)

,Class,Document,label2
0,space,series linked messages periodically posted att...,7
1,space,article according iau circular comet may tempo...,7


In [5]:
list_text= df["Document"].to_list()
list_labels=df["Class"].to_list()
list_labels2=df["label2"].to_list()

In [6]:
stratify_column_name = "label2"
dtrain= { "text": list_text, "label2": list_labels2, "label":list_labels}
dataset = Dataset.from_dict(mapping=dtrain, features=Features({"text":  Value(dtype='string', id=None), "label2": Value(dtype='int8'), 'label': Value(dtype='string', id=None)}))

dataset = dataset.shuffle(seed=42)
dataset = dataset.class_encode_column(stratify_column_name).train_test_split(test_size=0.3)
dataset = dataset.cast_column("label2", Clabels)

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label2', 'label'],
        num_rows: 700
    })
    test: Dataset({
        features: ['text', 'label2', 'label'],
        num_rows: 300
    })
})

In [8]:
# import pandas as pd
# from datasets import Dataset

# label_dic={"business":0,"entertainment":1,"food":2,"graphics":3,"historical":4,"medical":5,"politics":6,"space":7,"sport":8,"technologie":9}

# def load_dataset(data_path) -> Dataset:
#     """ Load dataset. """
#     stratify_column_name = "label2"
#     dataset_ecommerce_pandas = pd.read_csv(data_path, header=None, names=['label', 'text'])
#     dataset_ecommerce_pandas['label2']= dataset_ecommerce_pandas['label'].values
#     dataset_ecommerce_pandas['label'] = dataset_ecommerce_pandas['label'].astype(str)
#     dataset_ecommerce_pandas['label2'] = dataset_ecommerce_pandas['label2'].astype(str)
#     dataset_ecommerce_pandas['text'] = dataset_ecommerce_pandas['text'].astype(str)
#     dataset = Dataset.from_pandas(dataset_ecommerce_pandas)
#     dataset = dataset.shuffle(seed=42)
#     dataset = dataset.class_encode_column(stratify_column_name).train_test_split(test_size=0.3)
    
#     return dataset

In [9]:
# df = pd.read_csv("/kaggle/input/documents-raw/documents_raw.csv")
# df = df[["Class", "Document"]]
# df

In [10]:
# df.isnull().sum()

In [11]:
# data_path = "/kaggle/working/documents.csv"
# df.to_csv(data_path, index= False, header=None)

In [12]:
import evaluate
import nltk
import numpy as np
from typing import List, Tuple
from nltk.tokenize import sent_tokenize
from datasets import Dataset, concatenate_datasets
from huggingface_hub import HfFolder
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

MODEL_ID = "google/flan-t5-base"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
#dataset = load_dataset(data_path)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label2', 'label'],
        num_rows: 700
    })
    test: Dataset({
        features: ['text', 'label2', 'label'],
        num_rows: 300
    })
})

In [14]:
dataset['train'][1]

{'text': 'markets fall weak dollar oil prices sinking dollar hit shares monday finance meeting stern words fed chief alan london ftse fell nikkei 225 dropped steepest fall three g20 finance ministers said nothing supporting whose slide could jeopardise growth japan mr greenspan warned asian states could soon stop funding us monday euro close high dollar oil pushed higher investors fretted cold weather us europe potential output cut oil group although prices cooled end benchmark brent crude price closed 51 cents new york light sweet crude closed 25 cents slide comes us attempting talk traditional latest pitch president george w bush told asia pacific economic summit chile remained committed halving budget together trade red ink spreading across public finances widely seen key factor driving dollar last week us treasury secretary john snow told audience uk policy remained also said rate entirely markets signal traders took advice sell looked g20 meeting mr snow made clear exchange rates 

In [15]:
PROJECT = "FlanT5-Document-Classification"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "Kaggle-1000-Documents-10-classes"
# https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification
# business, entertainment,food, graphics, historical, medical, politics, space, sport, technologie


# Model
# https://huggingface.co/docs/transformers/model_doc/t5

##### https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

Classes : business, entertainment,food, graphics, historical, medical, politics, space, sport, technologie


# Model
https://huggingface.co/docs/transformers/model_doc/t5


*Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer*

https://arxiv.org/pdf/1910.10683.pdf

In [16]:
import wandb
with wandb.init(project=PROJECT, tags=[MODEL_NAME, DATASET],notes ="Fine tuning FlanT5 with Kaggle Dataset. 100 Documents 10 classes",job_type="dataset"): 
    wbtrain = wandb.Table(data=dataset['train'].to_pandas())
    wbtest = wandb.Table(data=dataset['test'].to_pandas())
    wandb.log({"dialogsum_train": wbtrain})
    wandb.log({"dialogsum_test": wbtest})


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [17]:
len(dataset['test'].to_pandas()['label'].unique())

10

In [18]:
dataset['test'].to_pandas()['label'].value_counts()

label
graphics         34
historical       32
entertainment    32
space            32
business         32
politics         30
medical          30
food             28
sport            28
technologie      22
Name: count, dtype: int64

In [19]:
dataset['train'].to_pandas()['label'].value_counts()

label
technologie      78
food             72
sport            72
politics         70
medical          70
entertainment    68
business         68
space            68
historical       68
graphics         66
Name: count, dtype: int64

In [20]:
# https://huggingface.co/google/flan-t5-base

# Autotokenizer

https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/auto#transformers.AutoTokenizer

In [21]:
# MODEL_ID = "google/flan-t5-base"
# # Load tokenizer of FLAN-t5
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [22]:
full = concatenate_datasets([dataset["train"], dataset["test"]])

In [23]:
full[:2]

{'text': ['celebrities get skates england footballer paul gascoigne join actress scarlett johnson bbc strictly ice christmas special also star television presenter carol smillie jessica taylor celebrity paired professional skater impress panel judges win audience bbc yet confirm final two stars battle become ice king presenter bruce forsyth tess daly host follows hot heels current saturday night series strictly come celebrities practise stipulated ice dance perform ice rink judges vote decide wins ice rink audience making rest show forms part festive ice skating duo jayne torvill christopher dean front similar celebrity ice dance show titled stars thin contestants stars thin ice paired professional skater learn new routine every end one celebrity crowned',
  'markets fall weak dollar oil prices sinking dollar hit shares monday finance meeting stern words fed chief alan london ftse fell nikkei 225 dropped steepest fall three g20 finance ministers said nothing supporting whose slide coul

In [24]:
# Metric
metric = evaluate.load("f1")

# # The maximum total input sequence length after tokenization.
# # Sequences longer than this will be truncated, sequences shorter will be padded.
# tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(
#     lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=[ 'label', 'label2']
# )
# max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
# print(f"Max source length: {max_source_length}")

# Tokenizers

large language models process text using tokens, which are common sequences of characters found in a set of text. The models learn to understand the statistical relationships between these tokens, and excel at producing the next token in a sequence of tokens.

https://platform.openai.com/tokenizer

In [25]:
# tokenizer("entertainment")

In [26]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
# tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
#     lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label2']
# )
# max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
# print(f"Max target length: {max_target_length}")

In [27]:
# tokenized_targets['input_ids'][:10], full['label'][:10]

In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [29]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
#model = T5ForConditionalGeneration.from_pretrained("t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [30]:
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=[ 'label', 'label2']
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

  0%|          | 0/1 [00:00<?, ?ba/s]

Max source length: 512


In [31]:
tokenizer("entertainment")

{'input_ids': [4527, 1], 'attention_mask': [1, 1]}

In [32]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label2']
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

  0%|          | 0/1 [00:00<?, ?ba/s]

Max target length: 3


In [33]:
tokenized_targets['input_ids'][:10], full['label'][:10]

([[4527, 1],
  [268, 1],
  [6525, 1],
  [6484, 1],
  [628, 1],
  [3, 13148, 1],
  [6484, 1],
  [6525, 1],
  [542, 1],
  [4527, 1]],
 ['entertainment',
  'business',
  'politics',
  'graphics',
  'space',
  'technologie',
  'graphics',
  'politics',
  'food',
  'entertainment'])

In [34]:
# cats = "business, entertainment,food, graphics, historical, medical, politics, space, sport, technologie"
# input_ids = tokenizer(f"Classify this text {dataset['train'][0]['text']} using one of these 10 categories {cats}", return_tensors="pt").input_ids
# labels = tokenizer(dataset["train"][0]['label'], return_tensors="pt").input_ids

In [35]:
# loss = model(input_ids=input_ids, labels=labels).loss

In [36]:
# loss.item()

In [37]:
REPOSITORY_ID = f"{MODEL_ID.split('/')[1]}-text-classification"

In [50]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=REPOSITORY_ID,
    per_device_train_batch_size=8,
    #per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,     # Overflows with fp16
    learning_rate=3e-4,
    num_train_epochs=10,
    logging_dir=f"{REPOSITORY_ID}/logs",    # logging & evaluation strategies
    logging_strategy="epoch",
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="wandb",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=REPOSITORY_ID,
    hub_token=user_secrets.get_secret("HF_TOKEN"),
)

def preprocess_function(sample: Dataset, max_target_length: int, padding: str = "max_length") -> dict:
    """ Preprocess the dataset. """

    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def postprocess_text(preds: List[str], labels: List[str]) -> Tuple[List[str], List[str]]:
    """ helper function to postprocess text"""
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 10) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [51]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label','label2'],fn_kwargs={"max_target_length": max_target_length})
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


# AutoModelForSeq2SeqLM

This is a generic model class that will be instantiated as one of the model classes of the library—with a sequence-to-sequence language modeling head

https://huggingface.co/docs/transformers/model_doc/t5

https://huggingface.co/transformers/v3.5.1/model_doc/auto.html#automodelforseq2seqlm

In [52]:

len(dataset['train'].features['label2'].names)


10

In [53]:
labels = dataset['train'].features['label2'].names
id2label={str(i): c for i, c in enumerate(labels)}
label2id={c: str(i) for i, c in enumerate(labels)}
id2label

{'0': 'business',
 '1': 'entertainment',
 '2': 'food',
 '3': 'graphics',
 '4': 'historical',
 '5': 'medical',
 '6': 'politics',
 '7': 'space',
 '8': 'sport',
 '9': 'technologie'}

In [54]:
label2id

{'business': '0',
 'entertainment': '1',
 'food': '2',
 'graphics': '3',
 'historical': '4',
 'medical': '5',
 'politics': '6',
 'space': '7',
 'sport': '8',
 'technologie': '9'}

In [43]:
# load model from the hub
# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID,    num_labels=len(labels),
#     id2label={str(i): c for i, c in enumerate(labels)},
#     label2id={c: str(i) for i, c in enumerate(labels)},)

In [55]:
model = T5ForConditionalGeneration.from_pretrained("t5-base", num_labels=len(labels),
     id2label={str(i): c for i, c in enumerate(labels)},
     label2id={c: str(i) for i, c in enumerate(labels)},)

In [56]:
nltk.download("punkt")

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [58]:
tokenized_dataset["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 300
})

In [59]:

import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [60]:
# TRAIN

with wandb.init(project=PROJECT, job_type="train"):
    train_results = trainer.train()
    trainer.save_model()

    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()
    print(train_results.metrics)

Step,Training Loss
88,0.755500
176,0.068500
264,0.034900
352,0.009600
440,0.001900
528,0.000200
616,0.000100
704,0.000700
792,0.000400
880,0.000100


***** train metrics *****
  epoch                    =       10.0
  total_flos               =  3969953GF
  train_loss               =     0.0872
  train_runtime            = 0:09:07.74
  train_samples_per_second =      12.78
  train_steps_per_second   =      1.607
{'train_runtime': 547.7484, 'train_samples_per_second': 12.78, 'train_steps_per_second': 1.607, 'total_flos': 4262705233920000.0, 'train_loss': 0.08717843088141473, 'epoch': 10.0}


train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▂▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,10.0
train/global_step,880


In [65]:

train_results

TrainOutput(global_step=880, training_loss=0.08717843088141473, metrics={'train_runtime': 547.7484, 'train_samples_per_second': 12.78, 'train_steps_per_second': 1.607, 'total_flos': 4262705233920000.0, 'train_loss': 0.08717843088141473, 'epoch': 10.0})

In [66]:

!mkdir flan-T5-fine-tune
custom_path = "./flan-T5-fine-tune/"
trainer.save_model(output_dir=custom_path)

In [67]:
with wandb.init(project=PROJECT, job_type="models"):
    artifact = wandb.Artifact("flan-T5-fine-tuned", type="model")
    artifact.add_dir(custom_path)
    wandb.save(custom_path)
    wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./flan-T5-fine-tune)... Done. 3.6s


In [70]:
tokenizer.save_pretrained(REPOSITORY_ID)

('flan-t5-base-text-classification/tokenizer_config.json',
 'flan-t5-base-text-classification/special_tokens_map.json',
 'flan-t5-base-text-classification/spiece.model',
 'flan-t5-base-text-classification/added_tokens.json')

In [ ]:
# tokenizer.save_pretrained(REPOSITORY_ID)
# trainer.create_model_card()
# trainer.push_to_hub()

In [71]:
import torch
from tqdm.auto import tqdm


from sklearn.metrics import classification_report



# Load model and tokenizer from the hub
#tokenizer = AutoTokenizer.from_pretrained(f"/kaggle/working/{REPOSITORY_ID}")
tokenizer = T5Tokenizer.from_pretrained(f"/kaggle/working/{REPOSITORY_ID}")

#model = AutoModelForSeq2SeqLM.from_pretrained(custom_path)
model = T5ForConditionalGeneration.from_pretrained(custom_path, num_labels=len(labels),
     id2label={str(i): c for i, c in enumerate(labels)},
     label2id={c: str(i) for i, c in enumerate(labels)},)

model = model.to('cuda') if torch.cuda.is_available() else model.to('cpu')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [73]:
def classify(text_to_classify: str) -> str:
    """Classify a text using the model."""
    inputs = tokenizer.encode_plus(text_to_classify, padding='max_length', max_length=512, return_tensors='pt')
    inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs.to('cpu')
    outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, early_stopping=True)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction


def evaluate_model() -> None:
    """Evaluate the model on the test dataset."""
    predictions_list, labels_list = [], []

    samples_number = len(dataset['test'])
    progress_bar = tqdm(range(samples_number))

    for i in range(samples_number):
        text = dataset['test']['text'][i]
        predictions_list.append(classify(text))
        labels_list.append(str(dataset['test']['label'][i]))

        progress_bar.update(1)

    report = classification_report(labels_list, predictions_list, zero_division=0)
    print(report)

In [74]:
 evaluate_model()

  0%|          | 0/300 [00:00<?, ?it/s]

               precision    recall  f1-score   support

     business       1.00      0.94      0.97        32
entertainment       1.00      0.97      0.98        32
         food       1.00      1.00      1.00        28
     graphics       0.94      1.00      0.97        34
   historical       1.00      0.97      0.98        32
      medical       1.00      0.93      0.97        30
     politics       1.00      1.00      1.00        30
        space       0.91      0.94      0.92        32
        sport       1.00      1.00      1.00        28
  technologie       0.88      1.00      0.94        22

     accuracy                           0.97       300
    macro avg       0.97      0.97      0.97       300
 weighted avg       0.98      0.97      0.97       300



In [75]:
text_to_classify ="""A 15-year-old boy has been arrested on suspicion of murdering Harry Pitman, who was fatally stabbed on New Year's Eve.

Harry, 16, from Haringey, was attacked in Primrose Hill, north London, at about 23:40 GMT on Sunday.

The boy was arrested - along with an 18-year-old man on suspicion of affray - on Tuesday night, the Met said.

Specialist detectives have found there was no indication the attack was racially motivated, the force added.

Vigil held for boy killed in New Year's Eve stabbing
Teen killed in New Year's Eve stabbing named
Det Ch Insp Geoff Grogan, who is leading the investigation, said despite having made the arrests, he was "still very keen to hear from anyone who has footage or information".

On Sunday, a 16-year-old boy was arrested at the scene on suspicion of murder and later released on bail pending further inquiries.
"""

In [76]:
tokens = text_to_classify.split(" ")
#tokens

In [77]:
tokens_pun_lower = [i.lower() for i in tokens if i.isalnum()]
#tokens_pun_lower

In [78]:
from nltk.corpus import stopwords
import nltk

stop_words = stopwords.words('english')
tokens_stop = [i for i in tokens_pun_lower if i not in stop_words]
#tokens_stop

In [79]:
text_to_class = " ".join(tokens_stop)
text_to_class

'boy arrested suspicion murdering harry fatally stabbed new attacked primrose north gmt boy arrested along man suspicion affray tuesday met detectives found indication attack racially force held boy killed new eve killed new eve stabbing ch insp geoff leading said despite made keen hear anyone footage boy arrested scene suspicion murder later released bail pending'

In [80]:
classify(text_to_class)

'politics'